# CP-467 Assignment 1

**Name: Charnel Dolon | ID: 212207670**

### Image Interpolation

1. Nearest neightbor interpolation implementation from scratch 

In [2]:
import cv2
import numpy as np

def nearest_neighbor_interpolation(image, scale_x, scale_y):
    # Get dimensions of the input image
    height, width = image.shape[:2]

    # Calculate the dimensions of the output image
    new_height = int(height * scale_y)
    new_width = int(width * scale_x)

    # Create an empty image for the output
    output_image = np.zeros((new_height, new_width, image.shape[2]), dtype=np.uint8)

    # Loop over every pixel in the output
    for l in range(new_height):
        for w in range(new_width):
            # Find the nearest pixel in the input
            x = int(w / scale_x)
            y = int(l / scale_y)
            # Assign the nearest pixel value to the output pixel
            output_image[l, w] = image[y, x]
    return output_image

# Load input image
image = cv2.imread('lena.tif')

# Downscale the image by half then upscale to original
downscaled_image = nearest_neighbor_interpolation(image, 0.5, 0.5)
upscaled_image = nearest_neighbor_interpolation(downscaled_image, 2.0, 2.0)

# Save the output image
cv2.imwrite('lena_nearest_scratch.tif', upscaled_image)

True

2. Nearest neighbor interpolation using OpenCV built-in function

In [14]:
import cv2
import numpy as np

image = cv2.imread('lena.tif')

# Downscale the image by half 
image_downscaled = cv2.resize(image, (image.shape[1]//2, image.shape[0] //2), interpolation=cv2.INTER_NEAREST)

# Upscale using Nearest Neighbor
image_upscaled_nearest = cv2.resize(image_downscaled, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
cv2.imwrite('lena_nearest_cv.tif', image_upscaled_nearest)

True

3. Bilinear interpolation implementation from scratch

In [13]:
def bilinear_interpolation(image, scale_x, scale_y):
    # Get dimensions of the input image
    height, width = image.shape[:2]

    # Calculate the dimensions of the output image
    new_height = int(height * scale_y)
    new_width = int(width * scale_x)

    # Create an empty output image
    output_image = np.zeros((new_height, new_width, image.shape[2]), dtype=np.uint8)

    # Loop over every pixel in the output image
    for l in range(new_height):
        for w in range(new_width):
            # Find the corresponding coordinates in the input image
            x = w / scale_x
            y = l / scale_y

            # Find the four nearest pixels 
            x1 = int(x)
            y1 = int(y)
            x2 = min(x1 + 1, width - 1)
            y2 = min(y1 + 1, height - 1)

            # Calculate the distances between the pixel and the four nearest pixels
            dx = x - x1
            dy = y - y1

            # Get the pixel values of each neighbor
            topleft = image[y1, x1]
            topright = image[y1, x2]
            bottomleft = image[y2, x1]
            bottomright = image[y2, x2]

            # Implement bilinear interpolation
            top = topleft * (1 - dx) + topright * dx
            bottom = bottomleft * (1 - dx) + bottomright * dx
            output_pixel = top * (1 - dy) + bottom * dy

            # Assign the calculated value to the output pixel
            output_image[l, w] = output_pixel

    return output_image

image = cv2.imread('lena.tif')

downscaled_image = bilinear_interpolation(image, 0.5, 0.5)
upscaled_image = bilinear_interpolation(downscaled_image, 2.0, 2.0)

cv2.imwrite('lena_bilinear_scratch.tif', upscaled_image)

True

4. Bilinear interpolation using OpenCV built-in function

In [5]:
import cv2
import numpy as np

image = cv2.imread('lena.tif')

image_downscaled = cv2.resize(image, (image.shape[1]//2, image.shape[0]//2), interpolation=cv2.INTER_LINEAR)

image_upscaled_bilinear = cv2.resize(image_downscaled, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_LINEAR)
cv2.imwrite('lena_bilinear_cv.tif', image_upscaled_bilinear)


True

5. Bicubic interpolation using OpenCV built-in function

In [4]:
import cv2

image = cv2.imread('lena.tif')

image_downscaled = cv2.resize(image, (image.shape[1]//2, image.shape[0]//2), interpolation=cv2.INTER_CUBIC)

image_upscaled_bicubic = cv2.resize(image_downscaled, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_CUBIC)
cv2.imwrite('lena_bicubic_cv.tif', image_upscaled_bicubic)

True

### Point Operations 

1. Find the negative of the image and store the output as "cameraman_negative".

In [20]:
import cv2

# Negative image (s = L - 1 - r)
def image_negative(image):
    negative_image = 255 - image
    return negative_image

image = cv2.imread('cameraman.tif')
cameraman_negative = image_negative(image)
cv2.imwrite('cameraman_negative.tif', cameraman_negative)

True

2. Apply power-law transformation on the image (experiment with different gamma
values) and store the (best) output as “cameraman_power”.

In [6]:
import cv2
import numpy as np

# (s = c*r^v)
def power_law_transformation(image, gamma, c=1):
    # Normalized image
    image_normalized = image / 255.0
    # Apply power-law transformation
    transformed_image = c * image_normalized**gamma
    # Scale back to the range [0, 255]
    transformed_image = np.uint8(transformed_image * 255)
    return transformed_image

image = cv2.imread('cameraman.tif')
gamma = 0.5  
cameraman_power = power_law_transformation(image, gamma)
cv2.imwrite('cameraman_power.tif', cameraman_power)

True

3. Apply bit-plane slicing on the image and store the outputs as “cameraman_b1”,
“cameraman_b2”, …, “cameraman_b8”.

In [22]:
import cv2
import numpy as np

def bit_plane_slicing(image, bit_plane):
    # Extract the specific bit plane by bitwise shifting and masking
    sliced_image = np.bitwise_and(image, 1 << bit_plane) >> bit_plane
    # Scale it back to 0-255
    sliced_image = sliced_image * 255
    return sliced_image

image = cv2.imread('cameraman.tif')
# Extract and save bit planes b1, b2, ..., b8
for bit in range(8):
    bit_plane_image = bit_plane_slicing(image, bit)
    cv2.imwrite(f'cameraman_b{bit+1}.tif', bit_plane_image)

### Histogram Processing 

1. Apply histogram equalization on the “einstein” image and store the output as
“einstein _equalized”.

In [8]:
import cv2
import numpy as np

def histogram_equalization_custom(image):
    # Flatten the image to 1D
    flat_image = image.flatten()
    # Find the histogram 
    hist = np.bincount(flat_image, minlength=256)
    # Number of pixels in the image 
    num_pixels = flat_image.size
    # Probability distribution function (PDF)
    pdf = hist / num_pixels
    # Cumulative distribution function (CDF)
    cdf = np.cumsum(pdf)
    # Find the maximum intensity value 
    max_intensity_value = np.max(image)
    # Scale the CDF by the maximum intensity value and round to nearest integer
    cdf_scaled = np.round(cdf * max_intensity_value).astype(np.uint8)
    # Map the original pixel values to the new equalized values and reshape
    equalized_image = cdf_scaled[flat_image]
    equalized_image = equalized_image.reshape(image.shape)
    return equalized_image

image = cv2.imread('einstein.tif')
# Apply histogram equalization using the custom function
equalized_image_custom = histogram_equalization_custom(image)

cv2.imwrite('einstein_equalized.jpeg', equalized_image_custom)

True

2. Apply histogram specification on “chest_x-ray1” image so that it matches the
histogram of “chest_x-ray2”. Store the output as “chest_x-ray3”.

In [47]:
import cv2
import numpy as np

def calculate_cdf(hist):
    # Compute the cumulative distribution function (CDF) from the histogram
    return np.cumsum(hist)

def histogram_specification(source_image, reference_image):
    # Flatten both images to 1D 
    source_flat = source_image.flatten()
    reference_flat = reference_image.flatten()

    # Calculate the histograms for both source and reference inputs
    source_hist = np.bincount(source_flat, minlength=256)
    reference_hist = np.bincount(reference_flat, minlength=256)

    # Compute the CDF for both 
    source_cdf = calculate_cdf(source_hist)
    reference_cdf = calculate_cdf(reference_hist)

    # Normalize the CDFs
    source_cdf_normalized = source_cdf * 255 / source_cdf[-1]
    reference_cdf_normalized = reference_cdf * 255 / reference_cdf[-1]

    # Create a mapping from source image values to reference image values
    mapping = np.zeros(256, dtype=np.uint8)
    ref_index = 0

    for src_index in range(256):
        while reference_cdf_normalized[ref_index] < source_cdf_normalized[src_index] and ref_index < 255:
            ref_index += 1
        mapping[src_index] = ref_index

    # Apply the mapping to the source image
    specified_image = mapping[source_flat]
    specified_image = np.reshape(specified_image, source_image.shape).astype(np.uint8)

    return specified_image

source_image = cv2.imread("C:/Users/Charnel/Desktop/Charnel's Folder/School/Y4S1/CP467/dolo7670_a01/chest_x-ray1.jpeg")
reference_image = cv2.imread("C:/Users/Charnel/Desktop/Charnel's Folder/School/Y4S1/CP467/dolo7670_a01/chest_x-ray2.jpeg")
# Apply histogram specification
specified_image = histogram_specification(source_image, reference_image)
cv2.imwrite('chest_xray3.jpeg', specified_image)

True